# 光学式文字認識

![A robot reading a newspaper](./images/ocr.jpg)

コンピュータビジョンでは、画像中の文字を検出して解釈することがよく行われます。このような処理は、**光学的文字認識**（OCR）と呼ばれています。

## Computer Visionサービスを利用した画像中の文字の読み取り

Computer Vision Cognitive Service は、以下のようなOCRタスクをサポートします。

- 複数の言語のテキストを読み取るために使用できる **OCR** API。このAPIは、同期して動作し、画像内の少量のテキストを検出して読み取る必要がある場合に有効です。

- 大きなドキュメントに最適化された **Read** API。このAPIは非同期で動作し、印刷されたテキストと手書きのテキストの両方に使用することができます。

これらのサービスを利用するには、**Computer Vision**リソースまたは**Cognitive Services**リソースを作成する必要があります。

まだCognitive Servicesリソースの作成を行っていない場合は、Azureサブスクリプションに**Cognitive Services**リソースを作成します。

> **補足**　すでにCognitive Servicesリソースがある場合は、Azureポータルでその**クイックスタート**ページを開き、そのキーとエンドポイントを下のセルにコピーするだけです。そうでない場合は、以下の手順で作成してください。

1. 新しいブラウザタブで、Azureポータル（https://portal.azure.com）を開き、Microsoftアカウントでサインインします。

2. **&#65291;リソースの作成** ボタンをクリックし、*Cognitive Services*を検索して以下の設定で **Cognitive Services** リソースを作成します。
    - **サブスクリプション**: *ご自身のサプスクリプション*
    - **リソースグループ**: *既存のリソースグループを選択するか、ユニークな名前で新しいリソースグループを作成します。*
    - **リージョン**: *利用可能なリージョンを選択（例:東日本）*
    - **名前**: *ユニークな名前を入力*
    - **価格レベル**: S0
    - **このボックスをオンにすることにより、以下のすべてのご契約条件を読み、同意したものとみなされます**: チェックを入れます。
3. デプロイが完了するまでしばらく待ちます。次に、Cognitive Servicesリソースにアクセスし、**Overview**ページで、サービスのキーを管理するリンクをクリックします。クライアントアプリケーションからCognitive Servicesリソースに接続するには、エンドポイントとキーが必要になります。

### Cognitive Servicesリソースのキーとエンドポイントの取得

Cognitive Servicesリソースを使用するためには、クライアントアプリケーションはそのエンドポイントと認証キーを必要とします。

1. Azureポータルで、Cognitive Servicesリソースの**キーとエンドポイント**ページを開き、リソースの**キー1**をコピーして、**YOUR_COG_KEY**を置き換えて、以下のコードに貼り付けます。

2. リソースの**エンドポイント**をコピーし、**YOUR_COG_ENDPOINT**を置き換えて、以下のコードに貼り付けます。

3. セルの実行<span>&#9655;</span>ボタン（セルの左上）をクリックして、下のセルのコードを実行します。

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_endpoint = 'YOUR_COG_ENDPOINT'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

キーとエンドポイントを設定したので、Computer Visionサービスのリソースを使って、画像からテキストを抽出することができます。

まず、**OCR** APIを使って、画像を同期的に分析し、含まれるテキストを読み取ることができます。このケースでは、架空の小売会社「Northwind Traders」の広告画像に、いくつかのテキストが含まれています。以下のセルを実行して、テキストを読んでみましょう。

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import os
%matplotlib inline

# Get a client for the computer vision service
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Read the image file
image_path = os.path.join('data', 'ocr', 'advert.jpg')
image_stream = open(image_path, "rb")

# Use the Computer Vision service to find text in the image
read_results = computervision_client.recognize_printed_text_in_stream(image_stream)

# Process the text line by line
for region in read_results.regions:
    for line in region.lines:

        # Read the words in the line of text
        line_text = ''
        for word in line.words:
            line_text += word.text + ' '
        print(line_text.rstrip())

# Open image to display it.
fig = plt.figure(figsize=(7, 7))
img = Image.open(image_path)
draw = ImageDraw.Draw(img)
plt.axis('off')
plt.imshow(img)

画像の中のテキストは、領域、行、単語の階層構造になっており、コードはこれらを読み取って結果を取得します。

結果では、画像の上に読み込まれたテキストを表示します。

## 境界ボックスの表示

結果には、画像の中にあるテキストの行や個々の単語の**境界ボックス**座標も含まれています。以下のセルを実行して、上記で取得した広告画像のテキスト行の境界ボックスを確認してください。

In [ ]:
# Open image to display it.
fig = plt.figure(figsize=(7, 7))
img = Image.open(image_path)
draw = ImageDraw.Draw(img)

# Process the text line by line
for region in read_results.regions:
    for line in region.lines:

        # Show the position of the line of text
        l,t,w,h = list(map(int, line.bounding_box.split(',')))
        draw.rectangle(((l,t), (l+w, t+h)), outline='magenta', width=5)

        # Read the words in the line of text
        line_text = ''
        for word in line.words:
            line_text += word.text + ' '
        print(line_text.rstrip())

# Show the image with the text locations highlighted
plt.axis('off')
plt.imshow(img)

出力結果には、テキストの各行の境界ボックスが画像上に長方形として表示されます。

## Read APIの使用

以前に使用したOCR APIは、少量のテキストを含む画像に対してはうまく機能します。スキャンされた文書など、より大きなテキストを読み取る必要がある場合は、 **Read** APIを使用することができます。これには、いくつかのステップが必要です。

1. Computer Visionサービスに画像を送信し、非同期的に読み取りと分析を行う。
   
2. 解析処理が完了するのを待つ。
   
3. 解析結果を取得します。

次のセルを実行して、このプロセスを使用して、Northwind Traders ストアのマネージャーに宛てたスキャンされた手紙のテキストを読み取ります。

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image
import time
import os
%matplotlib inline

# Read the image file
image_path = os.path.join('data', 'ocr', 'letter.jpg')
image_stream = open(image_path, "rb")

# Get a client for the computer vision service
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Submit a request to read printed text in the image and get the operation ID
read_operation = computervision_client.read_in_stream(image_stream,
                                                      raw=True)
operation_location = read_operation.headers["Operation-Location"]
operation_id = operation_location.split("/")[-1]

# Wait for the asynchronous operation to complete
while True:
    read_results = computervision_client.get_read_result(operation_id)
    if read_results.status not in [OperationStatusCodes.running]:
        break
    time.sleep(1)

# If the operation was successfuly, process the text line by line
if read_results.status == OperationStatusCodes.succeeded:
    for result in read_results.analyze_result.read_results:
        for line in result.lines:
            print(line.text)

# Open image and display it.
print('\n')
fig = plt.figure(figsize=(12,12))
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

結果を確認する。ほとんどが印刷されたテキストと手書きの署名で構成された手紙の完全なトランスクリプションがあります。OCR結果の下には、手紙のオリジナル画像が表示されています（スクロールしないと表示されない場合があります）。

## 手書きのテキストを読む

前述の例では、画像の解析要求で、**印刷された**テキストに最適化されたテキスト認識モードが指定されていました。それにもかかわらず、手書きの署名が読み取れたことに注目してください。

このように、手書きのテキストを読み取る機能は非常に便利です。例えば、買い物リストが書かれたメモを書き、スマホのアプリを使ってそのメモを読み、含まれているテキストを書き起こしたいとしましょう。

以下のセルを実行して、手書きの買い物リストの読み取り操作の例を見てみましょう。

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials
import matplotlib.pyplot as plt
from PIL import Image
import time
import os
%matplotlib inline

# Read the image file
image_path = os.path.join('data', 'ocr', 'note.jpg')
image_stream = open(image_path, "rb")

# Get a client for the computer vision service
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Submit a request to read printed text in the image and get the operation ID
read_operation = computervision_client.read_in_stream(image_stream,
                                                      raw=True)
operation_location = read_operation.headers["Operation-Location"]
operation_id = operation_location.split("/")[-1]

# Wait for the asynchronous operation to complete
while True:
    read_results = computervision_client.get_read_result(operation_id)
    if read_results.status not in [OperationStatusCodes.running]:
        break
    time.sleep(1)

# If the operation was successfuly, process the text line by line
if read_results.status == OperationStatusCodes.succeeded:
    for result in read_results.analyze_result.read_results:
        for line in result.lines:
            print(line.text)

# Open image and display it.
print('\n')
fig = plt.figure(figsize=(12,12))
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

## さらに学ぶ

OCRのComputer Visionサービスの使用についてより詳しい情報は、[光学式文字認識とは](https://docs.microsoft.com/ja-jp/azure/cognitive-services/computer-vision/concept-recognizing-text)を参照してください。